In [18]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

In [19]:
train = pd.read_csv('./output/f59.csv')
test = pd.read_csv('./output/f20.csv')
testid = test['sofifa_id']

In [20]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,36 mins 56 secs
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.1.1
H2O cluster version age:,3 days
H2O cluster name:,H2O_from_python_jose_h82qmr
H2O cluster total nodes:,1
H2O cluster free memory:,1.395 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [21]:
h2train = h2o.H2OFrame(train)
h2test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [22]:
columnas = [a for a in h2train.columns if a != 'overall']
x = columnas
y = 'overall'
print(columnas)

['sofifa_id', 'age', 'height_cm', 'weight_kg', 'nationality', 'club', 'potential', 'value_eur', 'wage_eur', 'player_positions', 'preferred_foot', 'international_reputation', 'weak_foot', 'skill_moves', 'work_rate', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_reflexes', 'gk_speed', 'gk_positioning', 'attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'defending_marking', 'defending_standing_tackle', 'defending_sliding_tackle', 'goalkee

H2oAutoML buscará automáticamente modelos de machine learning que se ajustan a los datos y entrenaremos con el mejor

In [23]:
automl = H2OAutoML(max_models=50, seed=42,max_runtime_secs=300, sort_metric='RMSE')
automl.train(x=x, y=y, training_frame=h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [28]:
print('[INFO] Models leader board:')
leader_board = automl.leaderboard
leader_board.head()

[INFO] Models leader board:


model_id,rmse,mean_residual_deviance,mse,mae,rmsle
XGBoost_1_AutoML_20200309_161526,0.552025,0.304731,0.304731,0.373386,0.00896859
StackedEnsemble_AllModels_AutoML_20200309_161526,0.55303,0.305842,0.305842,0.37311,0.00898936
XGBoost_2_AutoML_20200309_161526,20.7582,430.902,430.902,19.813,0.375525


In [29]:
metrics = automl.leader.cross_validation_metrics_summary().as_data_frame()
metrics

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,0.37338582,0.024799192,0.37316632,0.37292826,0.38597956,0.4008201,0.33403483
1,mean_residual_deviance,0.3047314,0.039310943,0.29487237,0.2981663,0.32625878,0.3549275,0.24943201
2,mse,0.3047314,0.039310943,0.29487237,0.2981663,0.32625878,0.3549275,0.24943201
3,r2,0.99401414,7.723026E-4,0.99421173,0.99414486,0.99359137,0.9930255,0.99509746
4,residual_deviance,0.3047314,0.039310943,0.29487237,0.2981663,0.32625878,0.3549275,0.24943201
5,rmse,0.5510896,0.035910945,0.5430215,0.5460461,0.57119066,0.5957579,0.49943167
6,rmsle,0.008952516,6.0020026E-4,0.008795345,0.008841463,0.009290352,0.00972001,0.00811541


In [30]:
automl.leader.varimp(use_pandas=True)

,variable,relative_importance,scaled_importance,percentage
0,value_eur,8.715824e+06,1.000000,0.361399
1,movement_reactions,6.212636e+06,0.712800,0.257605
2,potential,1.638571e+06,0.188000,0.067943
3,wage_eur,1.602630e+06,0.183876,0.066453
4,age,1.053111e+06,0.120828,0.043667
...,...,...,...,...
91,skill_moves,2.548589e+02,0.000029,0.000011
92,gk_speed,1.972508e+02,0.000023,0.000008
93,work_rate,1.872242e+02,0.000021,0.000008
94,weak_foot,1.767209e+02,0.000020,0.000007


In [32]:
h2o.save_model(automl.leader, path='./mymodel')

'/home/jose/proyecto ironhack/Soccer-Player-Predictor/mymodel/XGBoost_1_AutoML_20200309_161526'

In [33]:
model_path= '/home/jose/proyecto ironhack/Soccer-Player-Predictor/mymodel/XGBoost_1_AutoML_20200309_161526'
saved_model = h2o.load_model(model_path)

In [34]:
saved_model.predict(h2test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


predict
93.6513
93.0538
92.2648
89.836
91.1251
90.6673
89.6553
88.6376
89.0061
88.6433


In [10]:
type(automl.leader)

h2o.estimators.xgboost.H2OXGBoostEstimator

In [11]:
predictions = automl.leader.predict(h2test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [13]:
test['overall'] = predictions.as_data_frame()
columnasentrega = ['sofifa_id','overall']
entrega = test['overall']
test = test[columnasentrega]
test

,sofifa_id,overall
0,158023,93.334595
1,20801,92.799469
2,190871,92.543037
3,200389,90.172699
4,183277,91.046577
...,...,...
18273,245006,49.266388
18274,250995,49.542862
18275,252332,49.423573
18276,251110,48.024727


In [14]:
test.to_csv('output/predictfinal2.csv', index=False)

In [16]:
testing = pd.read_csv('output/predictfinal2.csv')

In [17]:
h2o.cluster().shutdown()

H2O session _sid_ab15 closed.


**XGBoost_1_AutoML_20200308_215025**


Este es el modelo que da mejores resultados